# A Guide for Applying Categorical Encoding Methods

In this notebook, we will be investigating the most common approaches to categorical encoding and how/when to apply them.

## Introduction

In applied machine learning, the two most common types of structured data are numeric data (such as `age`: 10, 17, 25) and categorical data (such as `color`: red, blue, green). 

It is often easier to deal with numeric data compared to categorical data, because machine learning models typically handle mathematical vectors--numeric data can therefore be much more directly applied.

However, machine learning algorithms cannot work directly with categorical data as they do not have intrinsic mathematical relations. Therefore, we must do some amount of work on the data before being able to use it in machine learning--the methods of turning categorical data into usable, mathematical data is called categorical encoding.

Feature engineering is one of the most manually intensive components of machine learning, and categorical encoding is one of the most common/time consuming parts of feature engineering. 

In this notebook, we will look at some of the most common approaches to categorical encoding.

## Categorical Encoding Flowchart

Use the flowchart below as a general overview to the different types of categorical encoders and when to use them.

![Categorical%20Encoding%20Flowchart.png](../flowchart/Categorical%20Encoding%20Flowchart.png)

We will explore the categorical encoding methods detailed in this flowchart through the [Featuretools categorical-encoding library](https://pypi.org/project/categorical-encoding/).

In [2]:
import pandas as pd
import categorical_encoding as ce
import utils_guide as utils

In [3]:
pd.options.display.float_format = '{:.2f}'.format #increase readability
feature_matrix, features, f1, f2, es, ids = utils.create_feature_matrix() #load in data for demos

## Identifying as Nominal vs. Categorical Data

#### Ordinal Data

Ordinal data are when the values within the category take on a meaningful ordering.

Examples of this include t-shirt sizes (`XS`, `S`, `M`, `L`, `XL`), survey opinions (`strongly dislike`, `dislike`, `like`, `strongly like`), or socieconomic status/income categories (`0-$50000`, `$50000-$100000`, `$100000+`).

#### Nominal Data
Nominal data have no meaningful ordering.

Examples of this include US States (`California`, `Massachusetts`, `New York`...), music genres (`Classical`, `Hip-hop`, `Jazz`...), or cuisine types (`Chinese`, `Italian`, `Tex-Mex`...).

## Classic Encoders

These encompass a broad range of encoders that are the most straightfoward and easiest to understand, making them very useful and popular among ML practioners.

### Ordinal/Label Encoding
In ordinal encoding, each string value is assigned a whole number specific to that value--the first unique value becomes 1, the second becomes 2, and so on.

As a quick example, our data will initially look like this.

In [4]:
feature_matrix

,product_id,value
id,,
0,coke zero,0.00
1,coke zero,5.00
2,coke zero,10.00
3,car,15.00
4,car,20.00
5,toothpaste,0.00


After fitting the Ordinal Encoder, it looks like this:

In [6]:
ce_ord = ce.Encoder(method='ordinal')
ce_ord.fit_transform(feature_matrix, features)

,PRODUCT_ID_ordinal,value
id,,
0,1,0.00
1,1,5.00
2,1,10.00
3,2,15.00
4,2,20.00
5,3,0.00


Ordinal Encoding can be useful in niche cases, namely for **interval data**. For example, if we had t-shirt sizes `[S,M,L]`, we could map them to `[1,2,3]` because t-shirt sizes follow a logical, evenly incrementing order.

However, keeping the data like this is usually not recommended, especially if the data values do not follow a regularly increasing order. Machine Learning algorithms cannot differentiate between categorical and numeric data and thus will infer an ordering that may be incorrect.

Thus, ordinal encoding is often less useful on its own. Instead, many encoders, such as [sklearn's OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) require data to be in a numeric format before the encoder can be applied. Then, most will use Ordinal Encoding as a first step before applying other encoders.

To alleviate this concern, Featuretools' categorical-encoding library's default encoders support direct encoding without having to first apply ordinal encoding.

### OneHot/Dummy Encoding

One-hot encoding is the go-to approach for categorical encoding due to its ease to use/understand, versatility, and accuracy. 

One-hot encoding works by creating a new column for each value. For each new column, a 1 is assigned if the row contains that column's value and a 0 otherwise.

In [7]:
ce_one_hot = ce.Encoder(method="one_hot")
ce_one_hot.fit_transform(feature_matrix, features)

,product_id = coke zero,product_id = car,product_id = toothpaste,value
id,,,,
0,1,0,0,0.00
1,1,0,0,5.00
2,1,0,0,10.00
3,0,1,0,15.00
4,0,1,0,20.00
5,0,0,1,0.00


One-hot encoding typically performs very well, and Featuretools' built-in `encode_features` features utilizes this. However, it has one major drawback.

The number of new features generated is equal to the number of unique values, which leads to severe memory issues with high cardinality datasets.

To illustrate, imagine if our data included 1000 unique products rather than 3. Then, we could go from our initial singular column to 1000 columns, one for each unique value. 

With so many added columns, memory issues can become a serious concern if coupled with many rows. There are also concerns with one-hot encoding when it comes to [decision-tree algorithms](https://roamanalytics.com/2016/10/28/are-categorical-variables-getting-lost-in-your-random-forests/). However, when dealing with the aforementioned memory issues, the fact that the encoded matrices are filled mostly with 0s suggests that there may be other alternative approaches.

### Binary Encoding
Binary encoding serves as an intermediary between one-hot encoding and ordinal encoding--it reduces the Ordinal's implicit ordering bias while creating fewer columns than one-hot.

In Binary Encoding, the categories' values are first Ordinal Encoded. The resulting integers are converted to binary, and then the resulting digits are split into columns.

In [9]:
ce_bin = ce.Encoder(method='binary')
ce_bin.fit_transform(feature_matrix, features)

,PRODUCT_ID_binary__0,PRODUCT_ID_binary__1,PRODUCT_ID_binary__2,value
id,,,,
0,0,0,1,0.00
1,0,0,1,5.00
2,0,0,1,10.00
3,0,1,0,15.00
4,0,1,0,20.00
5,0,1,1,0.00


Binary encoding can imply ordering, which can be either beneficial, detrimental, or negligble for model accuracy depending on the situation.

If one-hot encoding causes significant memory issues, binary encoding can serve as a simple, effective alternative that can reduce the problem.

### Hashing Encoding

Hashing Encoding also serves as a lower-dimensionality alternative to One-Hot encoding. Hashing Encoders employ the [hashing trick](https://medium.com/value-stream-design/introducing-one-of-the-best-hacks-in-machine-learning-the-hashing-trick-bf6a9c8af18f), which you can also read more about [here](https://booking.ai/dont-be-tricked-by-the-hashing-trick-192a6aae3087).

Hashing Encoders use a hashing algorithm to map category values to numeric values, which are then split into correspoding columns accordingly.

In [11]:
ce_hash = ce.Encoder(method='hashing')
ce_hash.fit_transform(feature_matrix, features)

,PRODUCT_ID_hashing__0,PRODUCT_ID_hashing__1,PRODUCT_ID_hashing__2,PRODUCT_ID_hashing__3,PRODUCT_ID_hashing__4,PRODUCT_ID_hashing__5,PRODUCT_ID_hashing__6,PRODUCT_ID_hashing__7,value
id,,,,,,,,,
0,0,0,0,0,1,0,0,0,0.00
1,0,0,0,0,1,0,0,0,5.00
2,0,0,0,0,1,0,0,0,10.00
3,0,1,0,0,0,0,0,0,15.00
4,0,1,0,0,0,0,0,0,20.00
5,0,0,0,1,0,0,0,0,0.00


The number of produced columns is a controllable parameter and can be set to be less than the number of unique values, meaning less total columns than one-hot encoding. The specific hashing algorithm is also controllable (default is `_md5_`).

Hashing Encoding presents its own unique challenge in the forming of collisions, but this does not usually result in problems unless there is significant overlap.

Overall, Hashing Encoding is another viable alternative in the case that one-hot encoding leads to dimensionality issues.

## Bayesian Encoders

Bayesian Encoders are different from Classic Encoders in that they use information from a dependent variable as well. They output only one column and thus eliminates any concern regarding high dimensionality.

### Target Encoding

Target Encoding replaces each specific category value with a weighted average of the dependent variable.

In [13]:
ce_targ = ce.Encoder(method='target')
ce_targ.fit_transform(feature_matrix, features, feature_matrix['value'])

,PRODUCT_ID_target,value
id,,
0,5.40,0.00
1,5.40,5.00
2,5.40,10.00
3,15.03,15.00
4,15.03,20.00
5,8.33,0.00


The primary concern with Target Encoding is overfitting/response leakage.

For example, if we were faced with the task of predicting `value` from `PRODUCT_ID_target`, information about `value` would have already been leaked via our number for `PRODUCT_ID_target`. 

With a little adjustment, however, these concerns can be alleviated.

### LeaveOneOut Encoding

LeaveOneOut Encoding is identical to TargetEncoding except it handles Target Encoding's problems with overfitting/response leakage.

In LeaveOneOut Encoding, the row in question leaves its own value out when calculating the mean.

In [15]:
ce_leave = ce.Encoder(method='leave_one_out')
ce_leave.fit_transform(feature_matrix, features, feature_matrix['value'])

,PRODUCT_ID_leave_one_out,value
id,,
0,7.50,0.00
1,5.00,5.00
2,2.50,10.00
3,20.00,15.00
4,15.00,20.00
5,8.33,0.00


Notice how each row has a different value because it does not include its own value in calculating the mean. This reduces label leakage, and, with a more substantial number of rows, the calculated mean should not vary greatly from category to category.

LeaveOneOut Encoding has no real drawbacks, but keep in mind that train/test data must be split before applying the encoder. Otherwise, information from the test data will leak into the training data.

## Alternative Encoders

The aforementioned encoders are the most commonly employed by machine learning practitioners, but other encoders exist for niche situations. We will run through several of them quickly.

### Additional Bayesian Encoders

#### Weights of Evidence

Weights of Evidence (WoE) tells the predictive power of an independent variable in relation to the dependent variable through the formula: $$\text{WoE} = \ln{\frac{\text{Distribution of non-events}}{\text{Distribution of events}}}.$$

WOE is especially useful in certain cases because similar WOE's imply similar categories, which could help with the accuracy of a machine learning algorithm.

Read more about WoE [here](https://www.listendata.com/2015/03/weight-of-evidence-woe-and-information.html).

#### James-Stein

The James-Stein estimator returns a weighted average of the global mean and of the local mean (specific to the particular category value).

This estimator was only designed for normal distributions. Read more about it [here](http://contrib.scikit-learn.org/categorical-encoding/jamesstein.html).

#### M-estimator

The M-Estimator performs similarly to TargetEncoding. Read more about it [here](http://contrib.scikit-learn.org/categorical-encoding/mestimate.html).

### Contrast Encoders

Contrast Encoders uses mathematical operations to capture differences/patterns between categories and order them accordingly.

Using Contrast Encoders is generally not advised as they produce a large number of output columns and generally do not outperform other encoders. However, in certain cases where categories follow a defined mathematical pattern, contrast encoders could offer better performance. 

Read this [guide](https://stats.idre.ucla.edu/r/library/r-library-contrast-coding-systems-for-categorical-variables/) to better understand the calculations behind the encoders. 

#### Helmert Encoding

Compares the mean of the dependent variable for a specific value to the mean of the dependent variable over all of the previous values.

#### Sum (Deviation) Encoding

Sum Encoding works the same as Helmert encoding except it compares the mean of the dependent variable to the overall mean over all of the levels instead of just the previous values.

#### Backward Difference

Similar to the previous two except the mean of the dependent variable is compared with the mean of only one level (the prior level).

#### Polynomial Difference

Polynomial encoding looks for linear, quadratic, cubic, or any degree trends. Interval Data, as mentioned earlier for Ordinal Encoding, is a specific subset of this (the values linearly increase).

## Summary

The go-to categorical encoding method should be one-hot encoding in nearly every scenario, with the exception of decision-tree based algorithms. It is straightforward to apply and typically performs well.

However, in the cases where one-hot encoding leads to memory issues, it is sometimes necessary to look to other encoders. Ordinal, Binary, Hashing, and Target encoders are all possible alternatives, although each presents its own unique set of benefits and drawbacks.

Another go-to method should be LeaveOneOut Encoding. It solves the memory issues that One-Hot Encoding raises, does not have the same concerns over response leakage as Target Encoding, and performs well with very little drawback in almost every situation.

Finally, Contrast Encoders provide an interesting way to mathematically separate categories and determine patterns. However, there are many concerns with contrast encoders, chiefly with its resulting high dimensionality issues as well as its lack of universal performance.

All in all, categorical encoding is an essential step in feature engineering/machine learning, and picking the correct method can be challenging. However, you should always feel free to test multiple categorical encoding methods and pick the one that yields in the best performance--this guide serves as a starting point to pick the right one.

## References/Additional Reading

A compilation of links that I found useful when writing this guide (in addition to the links already in the notebook).

#### Comparative Studies

https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf

http://www.willmcginnis.com/2015/11/29/beyond-one-hot-an-exploration-of-categorical-variables/

http://www.willmcginnis.com/2016/01/16/even-further-beyond-one-hot-hashing/


#### Useful Reading on Feature Engineering/Categorical Encoding

https://www.datacamp.com/community/tutorials/encoding-methodologies

https://towardsdatascience.com/understanding-feature-engineering-part-1-continuous-numeric-data-da4e47099a7b

https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63

https://towardsdatascience.com/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159

#### Relevant to Target Encoding

https://maxhalford.github.io/blog/target-encoding-done-the-right-way/

https://medium.com/datadriveninvestor/improve-your-classification-models-using-mean-target-encoding-a3d573df31e8

https://medium.com/datadriveninvestor/l1-l2-regularization-7f1b4fe948f2

#### Bayesian Encoding

https://stats.idre.ucla.edu/r/library/r-library-contrast-coding-systems-for-categorical-variables/#HELMERT

#### Machine Learning Application Examples

https://medium.com/airbnb-engineering/designing-machine-learning-models-7d0048249e69

#### BaseN Encoding

http://www.willmcginnis.com/2016/12/18/basen-encoding-grid-search-category_encoders/